In [258]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [259]:
!pip install pydub

In [0]:
# Imports 
from tensorflow.keras.models import load_model
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_audio
import os
import glob
import time
import importlib
from sklearn.preprocessing import StandardScaler
from subprocess import Popen, PIPE, STDOUT
os.system("pip install pydub")

os.chdir('/content/gdrive/My Drive/Machine-Learning-Projects/cs231n/notebooks/audio-new')


import arffToNp
importlib.reload(arffToNp)
import subprocess
import numpy as np


# Audio API

In [0]:

# TODO: add option for soft vs hard
def predict(mp4_filepath, best_model_filepath):
    """
    Outputs:
    - A tuple with predictions for each class (positive, neutral, negative)
    """

    model = fer_model()
    model.load_model(best_model_filepath)
    return model.predict(mp4_filepath)

class audio_model:
    def __init__(self):
        self.model = ()
        return

    def predict(self, mp4_filepath):
        self.preprocess(mp4_filepath)
        X = cv2.imload("test/happy.jpg")
        X = cv2.resize(X, (48,48))
        X = cv2.cvtColor(X, cv2.COLOR_BGR2GRAY)

        return self.model.predict(img)
        #return (0.1,0.2,0.7)

    def load_model(self, best_model_filepath):
        self.model = load_model(best_model_filepath)
        return

    def train(self, mp4_filepaths):
        # train the model
        # self.model = ....
        return

    def preprocess(self, mp4_filepath):

      """
      Outputs:
      - A numpy array with dimensions (m,n). 
        - m is the units in time dependent on the audio splice rate.
        - n is the number of features from the openSMILE library.
      """


      output_wav_file = mp4_filepath[-5] + 'extracted_audio.wav'
      mp4_filename = os.path.basename(mp4_filepath)
      audio_home_dir = os.path.dirname(mp4_filepath)
      os.chdir(audio_home_dir)

      # Strip the audio from video and store as .wav file
      ffmpeg_extract_audio(mp4_filepath, output_wav_file)
      !cd '$audio_home_dir' ; mkdir to_zip

      # splice the audio files into 2 seconds with 100 ms sliding window.
      # 30 kHz sampling rate
      !cd '$audio_home_dir' ; python SliceAudio.py -i *.wav -o wav -c 2  -b 2 -s 30000 -w 100 -l 2000
    
      # Zip and move files from drive to vm
      !cd '$audio_home_dir'  ; zip -r -qq to_zip.zip  to_zip ; cd '$audio_home_dir' ; mv 'to_zip.zip' '/content/' 

      # Remove the old zip folder in vm
      !cd '$audio_home_dir' ; cd to_zip ; rm *.wav  ; cd - ; rm -d to_zip

      # Inflate the zip folder in vm
      !cd '/content/' ; unzip -qq to_zip.zip 


      # OpenSMILE feature extraction
      out_fn = os.path.join(audio_home_dir, mp4_filename + 'openSmile-features.arff')
      os.chdir('/content/to_zip/')
      aligned_files = glob.glob('*.wav')
      os.chdir('/content/')
      for in_fn in aligned_files:
        in_fn = os.path.join('/content/to_zip/' , in_fn)
        name = os.path.basename(in_fn)
        !cd 'opensmile-2.3.0' ; inst/bin/SMILExtract -C config/IS13_ComParE.conf -I '$in_fn' -O '$out_fn' -N $name

      # Convert .arff to .csv
      all_timepoints_feature_array = arffToNp.convert(out_fn)
      print("The shape of the feature matrix for one \n video is: " , all_timepoints_feature_array.shape)

      # Clean up            
      !cd to_zip ; rm *.wav ; cd - ; rm -d to_zip      
      os.remove(out_fn)
      !cd '$audio_home_dir' ; rm *.wav ; rm *.csv
      !rm to_zip.zip


      # Standardize
      scaler = StandardScaler()
      all_timepoints_feature_array = scaler.fit_transform(all_timepoints_feature_array)

      return all_timepoints_feature_array


      # Read in each video file and add the (m,n) feature matrix to a 3D array

    def get_feature_batch(self, input_file_path , batch_size=3000):
        """
        Inputs: 
        - Path to the .mp4 files
        Outputs:
        - An ndarray with dims (s , m , n)
          - s is the number of samples
          - m is the number of slices for that sample (32)
          - n is the number of features (6373)
        """

        output_arr = []

        counter = 0

        for root, dir, files in os.walk(input_file_path):
          for name in files:
            if '.mp4' in name:

              file_path = os.path.join(input_file_path , name)
              print(file_path)

              one_sample_feat_matrix = self.preprocess(file_path)

              output_arr.append(one_sample_feat_matrix)

              if counter >= batch_size:
                break

              counter += 1




        return output_arr

def installOpenSMILE():
    """
    You must upload your downloaded version of openSMILE from the site to 
    cloud.

    """
    os.chdir('/content/')
    !tar -zxvf 'opensmile-2.3.0.tar.gz'
    !sed -i '117s/(char)/(unsigned char)/g' opensmile-2.3.0/src/include/core/vectorTransform.hpp
    !sudo apt-get update
    !sudo apt-get install autoconf automake libtool m4 gcc
    !cd 'opensmile-2.3.0' ; bash buildStandalone.sh



In [0]:
installOpenSMILE()


In [0]:
!cd 'opensmile-2.3.0' ; inst/bin/SMILExtract -h

In [0]:
audio_model_1 = audio_model()
output_arr = audio_model_1.preprocess(mp4_filepath='/content/gdrive/My Drive/Machine-Learning-Projects/cs231n/notebooks/audio-new/1_1.mp4')

In [230]:
!cd '/content/gdrive/My Drive/Machine-Learning-Projects/cs231n/notebooks/audio-new/'; ls 

1_1.mp4				  audio.ipynb		  openSMILEShell.sh
1extracted_audio.wav		  audio.py		  __pycache__
arffToNp.py			  opensmile-2.3.0	  SliceAudio.py
audio-deep-learning-models.ipynb  opensmile-2.3.0.tar.gz  to_zip


In [32]:
output_arr.shape

(32, 6373)

# Set up Processing Pipeline


In [0]:
# Imports

import tensorflow as tf
from tensorflow import keras


/bin/bash: line 0: cd: to_zip: No such file or directory
rm: cannot remove 'to_zip': No such file or directory


In [0]:
# Preprocessing Pipeling
!cd '$home_dir'; mkdir train_vids ; mkdir val_vids
train_path = '/content/gdrive/My Drive/Machine-Learning-Projects/cs231n/datasets/emotiw/Train.zip'
val_path = '/content/gdrive/My Drive/Machine-Learning-Projects/cs231n/datasets/emotiw/Val.zip'
vm_train_path = '/content/train_vids'
vm_val_path = '/content/val_vids'
slice_audio_path = '/content/gdrive/My Drive/Machine-Learning-Projects/cs231n/notebooks/audio-new/SliceAudio.py'

# Copy files to vm and inflate
!cp '$train_path' '$vm_train_path' ; cd '$vm_train_path' ; unzip -qq 'Train.zip'
!cp '$val_path' '$vm_val_path' ; cd '$vm_val_path' ; unzip -qq 'Val.zip'



In [0]:
inflated_train_path = '/content/train_vids/Train'
inflated_val_path = '/content/train_vids/Val'

# Copy SliceAudio to dirs
!cp '$slice_audio_path' '$inflated_train_path' ; cp '$slice_audio_path' '$inflated_val_path'

In [0]:
audio_model_test = audio_model()
      
batched_array = audio_model_test.get_feature_batch(inflated_train_path , batch_size=3)

In [253]:
final_array_1 = np.asarray(batched_array , dtype='float')
print(final_array_1.shape)

(3, 32, 6373)


/content
